<a href="https://colab.research.google.com/github/sophie-wjyang/Sales-Forecasting-System/blob/main/sales_forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -Uq upgini catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.6/159.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 79.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.3/108.3 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 446.6/446.6 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.4/109.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 913.9/913.9 kB 41.1 

Import libraries and files

In [ ]:
from os.path import exists
import pandas as pd

# import the file if it exists, or download it if it doesn't
df_path = "train.csv.zip" if exists("train.csv.zip") else "https://github.com/upgini/upgini/raw/main/notebooks/train.csv.zip"

Preprocess input data

In [ ]:

# convert it to a pandas dataframe
df = pd.read_csv(df_path)

# choose a random sample of 19000
df = df.sample(n=9_000, random_state=0)

# convert fields to strings
df["store"] = df["store"].astype(str)
df["item"] = df["item"].astype(str)

# convert field to date format and sort, for time series analysis
df["date"] = pd.to_datetime(df["date"])
df.sort_values("date", inplace=True)
df.reset_index(inplace=True, drop=True)
df.head()

,date,store,item,sales
0,2013-01-01,2,3,19
1,2013-01-01,10,21,33
2,2013-01-01,3,12,38
3,2013-01-01,3,27,11
4,2013-01-01,5,24,26


Split input data into training and test data

In [ ]:
train = df[df["date"] < "2017-01-01"]
test = df[df["date"] >= "2017-01-01"]

In [ ]:
# get the features and target of our training data
train_features = train.drop(columns=["sales"])
train_target = train["sales"]

test_features = test.drop(columns=["sales"])
test_target = test["sales"]

Find additional features

In [ ]:
from upgini import FeaturesEnricher, SearchKey
from upgini.metadata import CVType

enricher = FeaturesEnricher(
    search_keys = {
        "date": SearchKey.DATE,
    },
    cv = CVType.time_series
)

# search for additional features that are relevant to our dataset
enricher.fit(train_features, train_target, eval_set=[(test_features, test_target)])

[============================================================] 100% Finished


Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history

Detected task type: ModelTaskType.REGRESSION




Column name,Status,Errors
target,All valid,-
date,All valid,-



Running search request, search_id=91014179-2b76-46b6-9afa-dc6e5e7519d2
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com


57 relevant feature(s) found with the search keys: ['date']


f_weather_date_weather_umap_31_fa6d9a99,0.0572,100.0000,"4.7495, 4.9567, 4.7738",Upgini,Weather & climate normals data,Daily
f_weather_date_weather_umap_48_b39cd0c4,0.0265,100.0000,"5.1289, 4.6612, 4.936",Upgini,Weather & climate normals data,Daily
f_weather_date_weather_umap_34_c3ef5b4f,0.0197,100.0000,"5.5636, 5.5148, 5.585",Upgini,Weather & climate normals data,Daily
f_autofe_div_89c56a5f,0.0142,100.0000,"-1.5764, 0.486, -0.2185",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_mul_6a97c336,0.0068,100.0000,"-1.0908, 0.9333, -0.8468",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_mul_0cc09d2d,0.0055,100.0000,"55.7077, -85.7509, -13.8777",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_b97cf188,0.0051,85.0904,"0.2846, -1.437, 1.437",Upgini,AutoFE: features from Calendar data,Daily
f_weather_date_weather_umap_33_89bb7578,0.0048,100.0000,"4.7389, 4.9476, 5.2148",Upgini,Weather & climate normals data,Daily
f_autofe_mul_08e89348,0.0047,100.0000,"-19.3208, 41.914, -19.0518",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_mul_b59b15f6,0.0045,100.0000,"-7260.6927, 20008.7178, 7324.1...",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_20a12630,0.0044,100.0000,"-0.0224, 0.0013, 0.01",Upgini,"AutoFE: features from Calendar data,Markets data",Daily


Upgini,Weather & climate normals data,0.1234,13
Upgini,"AutoFE: features from Calendar data,Markets data",0.0781,27
Upgini,AutoFE: features from Markets data,0.0123,10
Upgini,AutoFE: features from Calendar data,0.0066,4
Upgini,Calendar data,0.0036,1
Upgini,Markets data,0.0014,1
Upgini,AutoFE: feature from Markets data,0.0006,1


"Calendar data,Markets data",f_autofe_div_89c56a5f,f_events_date_year_cos1_9014a856,f_financial_date_natural_gas_7d_to_7d_1y_shift_a5c3c07f,/
"Calendar data,Markets data",f_autofe_mul_6a97c336,f_events_date_week_cos1_f6a8c1fc,f_financial_date_vix_7d_to_1y_634c77eb,*
"Calendar data,Markets data",f_autofe_mul_0cc09d2d,f_events_date_year_cos1_9014a856,f_financial_date_crude_oil_1f195998,*
Calendar data,f_autofe_div_b97cf188,f_events_date_week_cos3_7525fe31,f_events_date_week_sin2_e28440a5,/
"Calendar data,Markets data",f_autofe_mul_08e89348,f_events_date_week_sin1_847b5db1,f_financial_date_crude_oil_1f195998,*
"Calendar data,Markets data",f_autofe_mul_b59b15f6,f_events_date_week_sin1_847b5db1,f_financial_date_dow_jones_65aaa996,*
"Calendar data,Markets data",f_autofe_div_20a12630,f_events_date_year_cos1_9014a856,f_financial_date_crude_oil_1f195998,/
"Calendar data,Markets data",f_autofe_div_3e22df83,f_events_date_week_sin1_847b5db1,f_financial_date_crude_oil_1f195998,/
"Calendar data,Markets data",f_autofe_div_aa0654f8,f_events_date_week_cos2_b0a07cfc,f_financial_date_usd_eur_7d_to_1y_200e4902,/
Markets data,f_autofe_div_35a59aed,f_financial_date_nasdaq_7d_to_1y_b00bfaa7,f_financial_date_silver_7d_to_1y_0ccfe462,/
"Calendar data,Markets data",f_autofe_div_9cf59d42,f_events_date_week_sin1_847b5db1,f_financial_date_dow_jones_65aaa996,/



Examples of outliers with maximum value of target:
37    205
21    196
43    176
Name: target, dtype: int64
Outliers will be excluded during the metrics calculation.
Calculating accuracy uplift after enrichment...

which makes metrics between the train and eval_set incomparable.


Train,7190,50.2833,315.2910,211.9242,103.3668
Eval 1,1810,59.5481,507.5848,392.7032,114.8816


Create an instance of the regressor model

In [ ]:
from catboost import CatBoostRegressor
from catboost.utils import eval_metric

# create an instance of the regressor model
model = CatBoostRegressor(verbose=False, allow_writing_files=False, random_state=0)

# estimate how much the enriched features will improve the accuracy of our model
enricher.calculate_metrics(
    train_features, train_target,
    eval_set=[(test_features, test_target)],
    estimator = model,
    scoring = "mean_absolute_percentage_error"
)

Calculating accuracy uplift after enrichment...
-
which makes metrics between the train and eval_set incomparable.


,Dataset type,Rows,Mean target,Baseline mean_absolute_percentage_error,Enriched mean_absolute_percentage_error,Uplift
0,Train,7190,50.2833,0.263530,0.169074,0.094455
1,Eval 1,1810,59.5481,0.267459,0.186788,0.080671


Add enriched features

In [ ]:
# enrich the dataset with additional features
enriched_train_features = enricher.transform(train_features, keep_input=True)
enriched_test_features = enricher.transform(test_features, keep_input=True)

           date store item
0    2013-01-01     2    3
1    2013-01-01    10   21
2    2013-01-01     3   12
3    2013-01-01     3   27
4    2013-01-01     5   24
...         ...   ...  ...
7185 2016-12-29    10   41
7186 2016-12-30     7   43
7187 2016-12-31     8   14
7188 2016-12-31     5   21
7189 2016-12-31     1   37

[7190 rows x 3 columns]


[============================================================] 100% Finished

You use Trial access to Upgini data enrichment. Limit for Trial: 10000 rows. You have already enriched: 1095 rows.

Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history



Column name,Status,Errors
date,All valid,-



Running search request, search_id=3399f0fd-1085-47f2-b09f-f12a54ee4158
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com

Retrieving selected features from data sources...


[============================================================] 100% Finished

You use Trial access to Upgini data enrichment. Limit for Trial: 10000 rows. You have already enriched: 2546 rows.

Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history



Column name,Status,Errors
date,All valid,-



Running search request, search_id=b3aacb0c-1660-4827-83ca-7b2d22c3eb5f
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com

Retrieving selected features from data sources...


Train the model with normal/enriched dataset

In [ ]:
# training the model with our original dataset (no enriched features)
model.fit(train_features, train_target)
predictions = model.predict(test_features)
eval_metric(test_target.values, predictions, "SMAPE")


[37.65141857448004]

In [ ]:
print(train_target)

0       19
1       33
2       38
3       11
4       26
        ..
7185    18
7186    22
7187    52
7188    21
7189    17
Name: sales, Length: 7190, dtype: int64


In [ ]:
# training the model with our enriched dataset
model.fit(enriched_train_features, train_target)
enriched_predictions = model.predict(enriched_test_features)
eval_metric(test_target.values, enriched_predictions, "SMAPE")

[61.23188143 66.30105867 56.39294302 ... 77.54848783 65.6723839
 69.6304011 ]


[16.15917409250358]